In [1]:
using JuMP
using Cbc

# Integer programming 

## 1. 

As the leader of an oil-exploration drilling venture, you must determine the least-cost selection of $5$ out of $10$ possible
sites. Label the sites $S_1, S_2, \dots , S_10,$ and the exploration costs associated with each as $C_1,C_2, . . . ,C_10$.

Regional development restrictions are such that:

i) Evaluating sites $S_1$ and $S_7$ will prevent you from exploring site $S_8$.
   
ii) Evaluating site $S_3$ or $S_4$ prevents you from assessing site $S_5$.
    
iii) Of the group $S_5, S_6, S_7, S_8,$ only two sites may be assessed.
    
Formulate an integer program to determine the minimum-cost exploration scheme that satisfies these restrictions.

__Solution:__

First we define $S_1, S_2, \dots, S_10$ as binary variables, if $S_i = 1$ the site $i$ is going to be select. Our objective function is to minimize the costs, it is:

$$\min {\sum_{i=1}^{10}} S_i C_i$$

We must select $5$ sites, so the sum of the binary variables must be $5$:

$$\sum_{i = 1}^{10}S_i = 5$$

In the first constraint we must have that if $S_1=1$ and $S_7=1 \implies S_8 = 0$. We can define this by:

$$S_8 \leq 2 - (S_1 + S_7)$$

If $S_1$ or $S_7$ is $0$ the constraint has no effect, as $S_8$ is binary.

In the second constraint we have an or situation, if $S_3=1$ or $S_4=1 \implies S_5 = 0$. We can represent by:

$$S_5 \leq 1 - S_3$$

$$S_5 \leq 1 - S_4$$

If $S_3$ or $S_4$ equals $1$, then we will have $S_5 \leq 0 \implies S_5 = 0$.

In the third constraint we have that only two variables (at max) from the set $\{S_5, S_6, S_7, S_8\}$ must be acessed. So:

$$S_5 + S_6 + S_7 + S_8 \leq 2 $$

Our final integer program is:

$$$$
$$
\begin{align}
& \min {\sum_{i=1}^{10}} S_i C_i \\
\text{subject to} & \\
& \sum_{i = 1}^{10}S_i = 5 \\
& S_8 \leq 2 - (S_1 + S_7) \\
& S_5 \leq 1 - S_3 \\
& S_5 \leq 1 - S_4 \\
& S_5 + S_6 + S_7 + S_8 \leq 2 \\
& S_i \geq 0, S_i \leq 1, i \in \{1, \dots, 10\} \\
& S_i \in \mathbb{Z}, i \in \{1, \dots, 10\} \\ 
\end{align}
$$

## 3.

The marketing group of A. J. Pitt Company is considering the options available for its next advertising campaign
program. After a great deal of work, the group has identified a selected number of options with the characteristics
shown in the accompanying table.


--- | TV | Trage Magazine| Newspaper | Radio | Popular Magazine | Promotional Magazine | Total resource available
--- | --- | --- | --- | --- | --- | --- | ---
Customers reached | 1,000,000 | 200,000 | 300,000 | 400,000 | 450,000 | 450,000 | ---
Cost($) | 500,000 | 150,000 | 300,000 | 250,000 | 250,000 | 100,000 | 1,800,000
Designers needed (man-hours) | 700 | 250 | 200 | 200 | 300 | 400 | 1,500
Salesmen needed (man-hours) | 200 | 100 | 100 | 100 | 100 | 1,000 | 1,200

The objective of the advertising program is to maximize the number of customers reached, subject to the
limitation of resources (money, designers, and salesman) given in the table above. In addition, the following
constraints have to be met:

i) If the promotional campaign is undertaken, it needs either a radio or a popular magazine campaign effort to
support it.

ii) The firm cannot advertise in both the trade and popular magazines.

Formulate an integer-programming model that will assist the company to select an appropriate advertising campaign
strategy.

__Solution__:

Initially we create variables $X_i, i \in \{1, 2, 3, 4, 5, 6\}$ that indicates if the campaign of type __{TV, Trage Magazine, Newspaper, Radio, Popular Magazine, Promotional Magazine}__ is choosen (in this order). $X_i$ are binary variables, and $X_i = 1$ indicates that the $i$-th campaign is choosen.

In [8]:
ad_campaign = Model(Cbc.Optimizer)
@variable(ad_campaign, X[i = 1:6], binary = true);

We need to create the first restrictions for the cost, the designers needed and the salesman needed. The cost for each campaign (in the order) is: $c = [\$500000, \$150000, \$300000, \$250000, \$250000, \$100000]$. The total money available is $\$1800000$. So our constraint will be:

$$\sum_{i = 1}^6 X_i c_i \leq 1800000$$

In [9]:
costs = [500000, 150000, 300000, 250000, 250000, 10000]
@constraint(ad_campaign, sum(X.*costs) <= 1800000);

The quantity of hours of work needed from the designers for each campaign is: $d =[700, 250, 200, 200, 300, 400]$. Our limit is $1500$ hours.

$$\sum_{i = 1}^6 X_i d_i \leq 1500$$

In [10]:
designers_hours = [700, 250, 200, 200, 300, 400]
@constraint(ad_campaign, sum(X.*designers_hours) <= 1500);

The quantity of hours of work needed from the salesmen for each campaign is: $s =[200, 100, 100, 100, 100, 1000]$. Our limit is $1200$ hours.

$$\sum_{i = 1}^6 X_i s_i \leq 1200$$

In [11]:
salesmen_hours = [200, 100, 100, 100, 100, 1000]
@constraint(ad_campaign, sum(X.*salesmen_hours) <= 1200);

Now, for the special constraints we will have from the first that if $X_6 = 1$ we must have that $X_4 = 1$ or $X_5 = 1$. We will represent that with:

$$X_6 \leq X_4 + X_5$$

In [12]:
@constraint(ad_campaign, X[6] <= X[4] + X[5]);

In the second constraint we can't have both $X_2$ and $X_5$ equals to $1$. So:

$$X_2 + X_5 \leq 1$$

In [13]:
@constraint(ad_campaign, X[2] + X[5] <= 1);

Our objective function will be to maximize the number of costumers reached. In our order, each campaign can reach the following number of costumers: $k = [1000000, 200000, 300000, 400000, 450000, 450000]$.

$$\max{\sum_{i = 1}^6 X_i k_i}$$

In [14]:
costumers = [1000000, 200000, 300000, 400000, 450000, 4500000]
@objective(ad_campaign, Max, sum(X.* costumers));

In [15]:
display(ad_campaign)

A JuMP Model
Maximization problem with:
Variables: 6
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 5 constraints
`VariableRef`-in-`MathOptInterface.ZeroOne`: 6 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)
Names registered in the model: X

In [16]:
optimize!(ad_campaign)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 5.5e+06 - 0.01 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0004I processed model has 4 rows, 6 columns (6 integer (6 of which binary)) and 17 elements
Cbc0012I Integer solution of -4950000 found by DiveCoefficient after 0 iterations and 0 nodes (0.03 seconds)
Cbc0012I Integer solution of -5350000 found by DiveCoefficient after 2 iterations and 0 nodes (0.04 seconds)
Cbc0013I At root node, 0 cuts changed objective from -5450000 to -5350000 in 2 passes
Cbc0014I Cut generator 0 (Probing) - 3 row cuts average 3.0 elements, 1 column cuts (1 active)  in 0.000 seconds - new frequency is 1
Cbc0014I Cut generator 1 (Gomory) - 0 row cuts average 0.0 elements, 0 column cuts (0 active)  in 0.000 seconds - new frequency is -100
Cbc0014I Cut generator 2 (Knapsack) - 0 row cuts average 0.0 el

This model is really simple and we can see the solution, the choosen campaigns:

In [31]:
campaigns = ["TV", "Trage Magazine", "Newspaper", "Radio", "Popular Magazine", "Promotional Magazine"]
println("Model solution:")
for i = 1:6
    if value.(X)[i] == 1.
        println("   " * campaigns[i])
    end
end
println("Total costumers reached:" * string(Int(objective_value(ad_campaign))))

Model solution:
   Radio
   Popular Magazine
   Promotional Magazine
Total costumers reached:5350000
